In [1]:
from llmtosql.model import WikiSQLModel
from llmtosql.trainer import Trainer
from llmtosql.dataloader import WikiSQLDataset
from llmtosql.utils.utils import plot_history, load_model, load_history
from matplotlib import pyplot as plt
import sys
import os
import json
import numpy as np
os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"
sys.path.insert(0, '..')

In [2]:
print(os.environ["VIRTUAL_ENV"])

/Users/albertomerango/Library/Caches/pypoetry/virtualenvs/models-J3z1PC2Q-py3.9


In [3]:
model = WikiSQLModel(base_model_type='bert-base-cased', attention_type='sqlnet', col_drop=True)
try:
    model = load_model(model, 'model_output/model.pth')
except Exception as e:
    print(f'No model loaded - {e}')

2023-03-03 23:29:11 [info     ] Using sqlnet attention mechanism


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
train_set =  WikiSQLDataset(type='dev', model=model)

In [ ]:
# val_set = WikiSQLDataset(type='dev')

In [ ]:
len(train_set.data)

In [ ]:
# datasets = (train_set, val_set)

In [ ]:
config = {
    'seed': 32,
    'scheduler': None,
    'optimizer': 'adam',
    'momentum': 0.9,
    'weight_decay': 0.0,
    'lr': 0.0005,
    'criterion': 'cross_entropy',
    'metric': 'accuracy',
    'pred_function': 'softmax',
    'model_dir': 'model_output',
    'backend': 'smddp'
}

In [ ]:
trainer = Trainer(model, datasets=train_set,
                  epochs=10, batch_size=32,
                  is_parallel=False,
                  **config)

In [ ]:
trainer.fit()

In [ ]:
trainer.save_history_('model_output')

In [ ]:
history = load_history('model_output')

In [ ]:
plot_history(history)

In [ ]:
model = MLModel()
model = load_model(model, 'model_output/model.pth')
test_loss, test_accuracy = trainer.test(model, trainer.val_loader)

In [ ]:
print(f'Model accuracy on test: {test_accuracy}')

In [4]:
train_set =  WikiSQLDataset(type='dev', model=model)

2023-03-03 23:29:22 [info     ] Tokenizing dataset.


100%|██████████| 8421/8421 [00:55<00:00, 150.77it/s]


In [5]:
for data in train_set:
    print(data['tokenized_inputs'])
    data['ss']

{'input_ids': tensor([[101, 160, 153,  ...,   0,   0,   0],
        [101, 177, 181,  ...,   0,   0,   0],
        [101, 170, 170,  ...,   0,   0,   0],
        ...,
        [101, 190, 181,  ...,   0,   0,   0],
        [101, 189, 190,  ...,   0,   0,   0],
        [101, 181, 171,  ...,   0,   0,   0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}


KeyError: 'ss'

In [6]:
from torch.utils.data import distributed, DataLoader

In [7]:
dataload = DataLoader(train_set, batch_size=4)

In [8]:
for sample_batched in dataload:
    print(list(sample_batched['input'][0]))

RuntimeError: stack expects each tensor to be equal size, but got [49, 512] at entry 0 and [45, 512] at entry 1